In [35]:
import os, sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

# 1. `Sequential` Model

## 1.1 Create a `Sequential` model
通过将一个网络层组成的列表传递给`Sequential`构造函数中的`layers`参数，可以创造一个`Sequential`模型；或通过`add()`方法将各层添加至`Sequential`对象的`.layers`属性中，进而建立模型；此外，通过`pop()`方法可以将模型最后一层删去

```python
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(16, activation="relu"),
])
# is equivalent to
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(16, activation="relu"))
len(model.layers)  # ==> 3

model.pop()
len(model.layers)  # ==> 2
```
可以看出，`Sequential`模型适用于模块的简单叠加，其中每个模块只有一个输入和一个输出；而不适用于有多个输入或输出、需要模块间共享权重、或搭建非线性结构模型的情况；

此外，`Sequential`构造函数同样地接受`name`参数，对于在 TBoard 中标注语义信息很有帮助


## 1.2 Specifying the input shape
作为`Sequential`模型的基本单元，Keras 网络层都需要知道它们输入的形状，以便创建相应的权重；然而大部分 Keras 构造函数并未将输入形状作为未知参数，进而当以`l1 = layers.Dense(3)`的方式创建网络层后，其权重并未被初始化，即`.weights`属性返回的是空列表；例如上面的示例中
```python
for l in seq_model.layers:
    assert len(l.weights) == 0
```

注意，此时调用模型的`.weights`属性会报错；网络层模型获取其权重的常见方式有 5 种：

1. 当一个网络层被添加至另一个网络层后面时，其会自动根据上一层网络的输出形状自动推断出本层的输入形状 (注意网络的输出形状总是位置函数)

2. 上述情况中，可能网络最前端的层模型仍无法获得其输入形状，此时可以利用`Input`对象指明，例如
```python
model = keras.Sequential()
model.add(keras.Input(shape=(64,)))
model.add(layers.Dense(16, activation="relu"))
len(model.weights)  # ==> 2
```

3. 或在整个模型搭建完成后，通过将一个张量作为输入送入网络，或调用`fit`等训练或验证的类方法，可以等效地指明了网络前端的输入形状；

4. 也可以在构建第一层时将`input_shape`或`input_dim`或`batch_input_shape`传递给`**kwargs`参数；需要注意的是，若在中间层指明输入形状时，该形状应与上一层输出形状匹配；示例如下
```python
model = keras.Sequential()
model.add(layers.Dense(16, input_shape=(4,)))
model.add(layers.Dense(32, input_dim=(16,)))
model.add(layers.Dense(10, batch_input_shape=(None, 32)))
len(model.weights)  # ==> 6
```

5. 调用`.build`方法初始化权重
```python
model = keras.Sequential()
model.add(layers.Dense(64))
model.add(layers.Dense(16))
model.build((None, 16))
len(model.weights)  # ==> 4
```
需要注意的是，在网络权重初始化之前，调用`.summary()`方法会报错

## 1.3 Transfer Learning with a Sequential model
迁移学习其中一步便是冻结底层模型而只训练顶层模型，更多有关迁移学习的内容参见[相关指导](https://www.tensorflow.org/guide/keras/transfer_learning/)；下面介绍 2 种利用模型进行迁移学习的方式；

1. 逐层指定`Sequential`模型中的网络层是否可训练，例如下面的例子中仅指定最后 2 层可训练：

```python
model = keras.Sequential([
    keras.Input(shape=(784))
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])
model.load_weights(...)

for layer in model.layers[:-2]:
    layer.trainable = False

model.compile(...)
model.fit(...)
```

2. 将预训练模型与一些模块通过`Sequential`构建为一个整体模型，例如

```python
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False
)
base_model.trainable = False

model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])
model.compile(...)
model.fit(...)
```

# 

# 

# 2. Functional API

相较于`Sequential`模型而言，Keras 的 functional API 可以搭建具有更加复杂网络拓扑的模型，具体而言，模型可以是非线性模型，模型间可以共享权重，并且可以有多个输入与输出；DL 中很大一部分模型是不同模块间的有向无环图 (Directed Acyclic Graph, DAG)，而 functional API 便是构建 DAG 的一种方法；




## 2.1 Manipulate complex graph topologies

### 2.1.1 Multiple inputs and outputs

利用 functional API 可以很容易处理多个输入和输出的情况；下面以一个模型为例演示多输入输出的实现方法，该模型能够按照优先级对顾客的问题票单进行排序，并将票单分派给指定部门；这个模型的输入包括票单标题、正文、用户特征，输出包括位于 (0, 1) 区间的优先级等数、被指派的部门；实现过程如下

```python
def rank_route_model(num_tags, size_voc, num_dep):
    title = keras.Input(shape=(None,), name="title")
    body = keras.Input(shape=(None,), name="body")
    tags = keras.Input(shape=(num_tags,), name="tags")
    # Embedding
    title_feat = layers.Embedding(num_words, 64)(title)  # 64-D vector
    body_feat = layers.Embedding(num_words, 64)(body)
    title_feat = layers.LSTM(128)(title_feat)  # 128-D vector
    body_feat = layers.LSTM(32)(body_feat)
    # merge and predict
    feat = layers.concatenate([title_feat, body_feat, tags])
    prior_pred = layers.Dense(1, name="priority")(feat)
    dep_pred = layers.Dense(num_dep, name="department")(feat)
    model = keras.Model(
        inputs=[title, body, tags],
        outputs=[prior_pred, dep_pred],
    )
    return model

model = rank_route_model(12, 10000, 4)
keras.utils.plot_model(
    model,
    "../../test/Guide/Keras/01.Create_a_Model/multi_in_out.png",
    show_shapes=True,
    dpi=512
)
```

<img src="../../test/Guide/Keras/01.Create_a_Model/multi_in_out.png" width=720>


















在对模型进行编译时，可以为每个输出分配不同类型的损失，以及为每个损失分配不同的权重；在输出层指明了名称的情况下，可以以字典而非列表的形式将损失类型传递给`loss`参数，`fit()`方法也是如此；

```python
# Dummy data
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, 12)).astype("float32")
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, 4))

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "priority": BinaryCrossentropy(from_logits=True),
        "department": CategoricalCrossentropy(from_logits=True),
    },
    loss_weights=[1.0, 0.2],
)
model.fit(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_targets, "department": dept_targets},
    epochs=2,
    batch_size=32,
)
```
更多与训练有关的细节参见[相关指导](https://www.tensorflow.org/guide/keras/train_and_evaluate/)



### 2.1.2 Non-linear connectivity topologies
最常见的非线性拓扑便是残差结构，可参见下面示例中的通过 functional API 所实现 ResNet 模型的残差单元：

```python
def res_unit(in_shape, num_filter):
    stride = int(num_filter / in_shape[-1])
    shortcut = inputs = keras.Input(shape=in_shape)
    x = layers.Conv2D(
        num_filter, (3, 3), strides=stride,
        padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(
        num_filter, (3, 3), strides=1,
        padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if stride != 1:
        shortcut = layers.Conv2D(
            num_filter, (1, 1), strides=stride,
            padding='same', use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    outputs = layers.Activation("relu")(x + shortcut)
    return keras.Model(inputs=inputs, outputs=outputs)

def res_block(unit, num_unit, in_shape, num_filter):
    x = inputs = keras.Input(shape=in_shape)
    for _ in range(num_unit):
        x = unit(x.shape[-3:], num_filter)(x)
    return keras.Model(inputs=inputs, outputs=x)

def resnet(unit, num_blocks, in_shape=(32, 32, 3), num_classes=10):
    inputs = keras.Input(shape=in_shape)
    x = layers.Conv2D(
        16, (3, 3), strides=1,
        padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = res_block(unit, num_blocks[0], x.shape[-3:], 16)(x)
    x = res_block(unit, num_blocks[1], x.shape[-3:], 32)(x)
    x = res_block(unit, num_blocks[2], x.shape[-3:], 64)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(num_classes, use_bias=False)(x)
    return keras.Model(inputs=inputs, outputs=outputs)
    
def resnet20():
    return resnet(res_unit, [3, 3, 3])

model = resnet20()
res_block = model.get_layer(index=5)
res_unit = res_block.get_layer(index=1)
keras.utils.plot_model(
    model=res_unit,
    "../../test/Guide/Keras/01.Create_a_Model/functional_api_resnet20_resUnit.png",
    show_shapes=True
)
```

<img src="../../test/Guide/Keras/01.Create_a_Model/functional_api_resnet20_resUnit.png">








### 2.1.3 Shared layers

通过 functional API 可以很容易实现共享网络层；共享网络层指在同一模型中重复使用某些层的实例，这些层通常用于编码相似空间的输入，例如具有相似词汇表的两段不同文本；通过在不同的输入之间共享信息，可以减少训练模型所需的数据；

要通过 functional API 共享网络层，只需多次调用同一个层模型的实例，例如下面的示例

```python
# Embedding for 1000 unique words mapped to 128-dimensional vectors
shared_embedding = layers.Embedding(1000, 128)
text_input_a = keras.Input(shape=(None,), dtype="int32")
text_input_b = keras.Input(shape=(None,), dtype="int32")
# Reuse to encode both inputs
encoded_input_a = shared_embedding(text_input_a)
encoded_input_b = shared_embedding(text_input_b)
```





## 2.2 Extract and reuse nodes in the graph of layers
由网络层构成的图是静态数据结构，进而可以对其进行访问及检查，例如通过访问将模型绘制成计算图，或者将中间层的输出用于其他任务上；参见下面提取 VGG-19 不同层的输出特征的模型的示例：

```python
vgg19 = tf.keras.applications.VGG19()
features_list = [layer.output for layer in vgg19.layers]
feat_extractor = keras.Model(inputs=vgg19.input, outputs=features_list)
img = np.random.random((1, 224, 224, 3)).astype("float32")
extracted_features = feat_extraction_model(img)
```

该技术常用于风格转换等任务中；






## 2.3 Customize a functional API
自定义 Functional API 的详细方式可参见下面类继承的 [3.1 小节](#3.1-Subclassing-Layer-class)，这里仅不加说明的提供一个自定义全连接层的示例

```python
class Linear(layers.Layer):
    def __init__(self, out_dim=32):
        super(Linear, self).__init__()
        self.out_dim = out_dim

    def build(self, in_shape):
        self.w = self.add_weight(
            shape=(in_shape[-1], self.out_dim),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.out_dim,),
            initializer="random_normal",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {"out_dim": self.out_dim}

inputs = keras.Input((4,))
outputs = Linear(10)(inputs)
model = keras.Model(inputs, outputs)
config = model.get_config()
new_model = keras.Model.from_config(
    config, custom_objects={"out_dim": out_dim}
)
```





## 2.4 Pros and cons of functional API
- **优点**
    - 可以在定义连接图时对模型进行检查；<br>
    由于使用 functional API 时，输入形状和数据类型均会预先声明，进而每个附加层均会检查传递给它的张量形状和数据类型是否与其自身所指定的参数相匹配；利用这种机制，除与模型收敛有关的调试外，其他调试均可以在构建模型期间完成；<br>
    - functional 模型便于序列化及复制；<br>
    由于functional 模型是一种数据结构而非代码，进而其可以直接被序列化并以 SavedModel 格式保存；然而继承模型则需要人工实现`get_config()`和`from_config()`方法；<br>
    - functional 模型便于绘制和检查，如 2.2 节所述


- **缺点**<br>
    - functional API 无法实现动态网络架构；functional API 可以处理由层模型构成的有向无环图，而无法实现递归网络或 Tree-RNN 等模型，进而必须通过继承模型实现

更多关于 functional API 和继承模型之间的区别，请参见 [TF2.x 中的符号 API 和命令式 API](https://blog.tensorflow.org/2019/01/what-are-symbolic-and-imperative-apis.html)



# 

# 

# 3. Subclassing
## 3.1 Subclassing `Layer` class
`Layer`类封装了一个层的状态(也称为一个层的权重)和从输入到输出的映射；下面是手动实现全连接层的示例：
```python
class Linear(keras.layers.Layer):
    def __init__(self, out_dim=32, in_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            w_init(shape=(in_dim, out_dim), dtype="float32"),
            trainable=True
        )
        # or via a quicker shortcut to add weight
        self.b = self.add_weight(
            shape=(out_dim,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

linear = Linear()
assert len(linear.trainable_weights) == 2
assert len(linear.non_trainable_weights) == 0
```
这里`w`和`b`均指定了`Trainable=True`，进而该层会自动追踪这两个变量；此外，也可以通过指定`Trainable=False`以使得需要的变量在梯度下降时保持不变，例如 BN 中的均值和方差；利用`trainable_weights`和`non_trainable_weights`属性可以查看该层的可训练和不可训练权重，示例如上；

### 3.1.1 Deferring weight creation
由于大多情况下无法获知输入的形状，以类似`Dense`、`Conv2D`等 API 那样将权重的创建进行延迟是一个不错的选择，延迟创建可以通过重写`.build()`方法实现；其内部机制在于调用`call()`方法时会先调用`.build()`方法，进而完成对权重的初始化，上面的示例可以改为如下的实现方式：

```python
class Linear(keras.layers.Layer):
    def __init__(self, out_dim=32):
        super(Linear, self).__init__()
        self.out_dim = out_dim

    def build(self, in_shape):
        self.w = self.add_weight(
            shape=(in_shape[-1], self.out_dim),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.out_dim,),
            initializer="random_normal",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
```
若在自定义层对象中使用了 keras 模块内置的层模型，考虑到这些内置的层模型已经实现了`build`方法，进而可以直接在`__init__()`方法中对其进行创建

### 3.1.2 The `add_loss()` method
通过调用`.add_loss(value)`可以创建训练时使用的损失；该层的损失及其内部层所定义的损失会被附加到`Layer.losses`属性，这些损失会在每次调用`__call__()`方法时重置，进而保证`losses`只含有上次前向传播的损失；

```python
class Activ_Regul(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(Activ_Regul, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

class Kernel_Regul(keras.layers.Layer):
    def __init__(self):
        super(Kernel_Regul, self).__init__()
        self.activ_regul = Activ_Regul(1e-2)
        self.dense = keras.layers.Dense(
            32, kernel_regularizer=keras.regularizers.l2(1e-3)
        )

    def call(self, inputs):
        self.activ_regul(inputs)
        return self.dense(inputs)

layer = Kernel_Regul()
_ = layer(tf.ones([3, 2]))
assert len(layer.losses) == 2
```
通过`add_loss()`方法声明的损失会在调用`Model.fit()`方法时自动计入总损失内 (需要注意的是，`Layer`并不含有`fit`方法)；然而若想要人工编写训练循环，则需要显示地将`losses`中的损失添加至总损失内，可参见下面的示例；
```python
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
for x_train, y_train in train_dataset:
    with tf.GradientTape() as tape:
        y_pred = layer(x_train)
        loss = loss_fn(y_train, y_pred)
        loss += sum(model.losses)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
```
更多关于人工实现训练循环的细节，可参见[相关指导](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch/)

### 3.1.3 The `add_metric()` method
`Layer`会在整个训练过程中对`add_metric()`方法添加的指标进行追踪，并将该指标的平均值添加至`Layer.metrics`属性中；与损失相同，`Model.fit()`同样也会对该指标进行追踪，可参见下面的示例：
```python
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.acc_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        loss = self.loss_fn(targets, logits, sample_weights)
        acc = self.acc_fn(targets, logits, sample_weights)
        self.add_loss(loss)
        self.add_metric(acc, name="accuracy")
        return tf.nn.softmax(logits)

inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)
model = keras.Model(inputs=[inputs, targets], outputs=predictions)

model.compile(optimizer="adam")
data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)
```
### 3.1.3 Priviliged arguments in `call()` method
- `training`

    某些层例如 BN 和 Dropout 等，其在训练和推理过程中执行的操作并不相同，进而需要在`call()`方法中暴露一个`training`参数，以便于`fit()`方法实现训练-验证循环时能够正确地在相应模式下进行计算；一个 Dropout 层的实现示例如下：
    ```python
    class CustomDropout(keras.layers.Layer):
        def __init__(self, rate, **kwargs):
            super(CustomDropout, self).__init__(**kwargs)
            self.rate = rate

        def call(self, inputs, training=None):
            if training:
                return tf.nn.dropout(inputs, rate=self.rate)
            return inputs
    ```
- `mask`

    `mask`参数常用于 RNN 模型；`mask`是一个布尔张量，其每个元素对应时间序列在一个时间步的输入，用于指明该时间步下是否对输入进行掩码操作；在上一层生成掩码后，Keras 会自动将正确的`mask`参数传递给支持该参数的层模型的`__call__()`方法；可以生成掩码的层包括`Embedding`、`Masking`等；更多关于如何自定义一个支持掩码的层模型的方法，可参见[相关指导](https://www.tensorflow.org/guide/keras/masking_and_padding/)
    

### 3.1.4 Optionally enable serialization
通过实现`get_config()`方法可以使自定义的层对象变得可序列化；值得一提的是，`Layer`类的初始化函数还接收一些关键字参数，例如`name`、`dtype`、`trainable`等，进而最好在`__init__()`方法中将这些参数传递给父类，同时将其写入`get_config`方法中；示例如下

```python
class Linear(layers.Layer):
    def __init__(self, out_dim=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.out_dim = out_dim

    def build(self, in_shape):
        self.w = self.add_weight(
            shape=(in_shape[-1], self.out_dim),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.out_dim,),
            initializer="random_normal",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"out_dim": self.out_dim})
        return config


layer = Linear(64)
config = layer.get_config()
new_layer = Linear.from_config(config)
```
如果需要对从`config`中反序列化的过程作更多细节要求，可以尝试重写`from_config()`方法，下面是该方法的基本实现
```python
def from_config(cls, config):
    return cls(**config)
```
更多有关模型序列化及保存的内容，可参见[相关指导](http://localhost:8888/tree/Help_Viewer_Python/TensorFlow/TF2/Guide/Save_a_Model)

## 3.2 The `Model` class
官方建议使用`Layer`类定义内部模块块，使用`Model`类定义外部模型；例如对于 ResNet-50 而言，残差单元及其堆叠形成的模块可以继承`Layer`类，整个`ResNet50`模型继承`Model`类；

`Model`类具有所有与`Layer`类所拥有的 API，但与`Layer`类不同的是，`Model`类还暴露了`fit()`、`evaluation()`、`predict()`、`save()`、`save_weights()`方法 (注意以上代码均将继承`Layer`类的对象作为 functional API 使用，即该对象本身不含有与训练验证等相关方法)，以及能够显示其内部层结构的属性`layers`；进而如果使用者需要调用以上方法时，则应继承`Model`类；若当前模型仅仅是一个更大模型的子模块，或者需要手动编写训练和保存模型的代码，可以使用`Layer`类；


## 3.3 Putting it all together: an end-to-end example

下面以变分自编码器 (Variational AutoEncoder, VAE) 为例，在 MNIST 数据集上示范一个端到端的训练过程，其中 VAE 继承`Model`类，其内部模块继承`Layer`类，损失为正则化损失（即 KL 散度）

In [132]:
import tensorflow.data as data

class Sampling(layers.Layer):
    """Use (z_mean, z_log_var) to sample z, the vector encoding a digit"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch, dim = z_mean.shape[0:2]
        epsilon = keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps img vector to a triplet (z_mean, z_log_var, z)"""

    def __init__(self, latent_dim=32, inter_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(inter_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts the encoded vector back into a readable digit."""

    def __init__(self, orig_dim, inter_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(inter_dim, activation="relu")
        self.dense_output = layers.Dense(orig_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    def __init__(self, orig_dim, inter_dim=64, latent_dim=32, name="autoencoder", **kwargs):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.orig_dim = orig_dim
        self.encoder = Encoder(latent_dim, inter_dim)
        self.decoder = Decoder(orig_dim, inter_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed

optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.MeanSquaredError()
metric = keras.metrics.Mean()
vae = VariationalAutoEncoder(784, 64, 32)

(x_train, _), _ = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255.
train_dataset = data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

for epoch in range(2):
    print("==> Epoch %d" % (epoch,))
    for step, inputs in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(inputs)
            loss = loss_fn(inputs, reconstructed)
            loss += sum(vae.losses)
        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        metric(loss)
        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, metric.result()))

==> Epoch 0
step 0: mean loss = 0.3572
step 100: mean loss = 0.1271
step 200: mean loss = 0.1001
step 300: mean loss = 0.0897
step 400: mean loss = 0.0847
step 500: mean loss = 0.0813
step 600: mean loss = 0.0791
step 700: mean loss = 0.0774
step 800: mean loss = 0.0762
step 900: mean loss = 0.0752
==> Epoch 1
step 0: mean loss = 0.0749
step 100: mean loss = 0.0742
step 200: mean loss = 0.0737
step 300: mean loss = 0.0732
step 400: mean loss = 0.0728
step 500: mean loss = 0.0724
step 600: mean loss = 0.0721
step 700: mean loss = 0.0718
step 800: mean loss = 0.0716
step 900: mean loss = 0.0713
